In [1]:
import numpy as np
import h5py
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from collections.abc import Iterable
import time
import math
import PyFiles.MLFunctionsForPytorch as MLFun
import pandas as pd
batchSize = 128 #Batch size of training set

# Define our neural networks

In [2]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.norm0 = nn.BatchNorm1d(3)
    self.linear1 = nn.Linear(in_features=3, out_features=64)
    self.norm1 = nn.BatchNorm1d(64)
    self.act1 = nn.LeakyReLU()
    self.linear2 = nn.Linear(in_features=64, out_features=16)
    self.norm2 = nn.BatchNorm1d(16)
    self.act2 = nn.LeakyReLU()
    self.output = nn.Linear(in_features=16, out_features = 3)
    


  def forward(self, x):
    '''
      Forward pass
    '''
    x = self.norm0(x)
    x = self.linear1(x)
    x = self.norm1(x)
    x = self.act1(x)
    x = self.linear2(x)
    x = self.norm2(x)
    x = self.act2(x)
    x = self.output(x)
    
    
    return x

class MLP2Input(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.norm0 = nn.BatchNorm1d(2)
    self.linear1 = nn.Linear(in_features=2, out_features=64)
    self.norm1 = nn.BatchNorm1d(64)
    self.act1 = nn.LeakyReLU()
    self.linear2 = nn.Linear(in_features=64, out_features=16)
    self.norm2 = nn.BatchNorm1d(16)
    self.act2 = nn.LeakyReLU()
    self.output = nn.Linear(in_features=16, out_features = 3)
    


  def forward(self, x):
    '''
      Forward pass
    '''
    x = self.norm0(x)
    x = self.linear1(x)
    x = self.norm1(x)
    x = self.act1(x)
    x = self.linear2(x)
    x = self.norm2(x)
    x = self.act2(x)
    x = self.output(x)
    
    
    return x

class MLP3Input(nn.Module):
    '''
    Multilayer Perceptron for regression.
    '''
    def __init__(self):
        super().__init__()
        self.norm0 = nn.BatchNorm1d(3)
        self.linear1 = nn.Linear(in_features=3, out_features=256)
        self.norm1 = nn.BatchNorm1d(256)
        self.act1 = nn.LeakyReLU()
        self.linear2 = nn.Linear(in_features=256, out_features=128)
        self.norm2 = nn.BatchNorm1d(128)
        self.act2 = nn.LeakyReLU()
        self.linear3 = nn.Linear(in_features=128, out_features=64)
        self.norm3 = nn.BatchNorm1d(64)
        self.act3 = nn.LeakyReLU()
        self.linear4 = nn.Linear(in_features=64, out_features=32)
        self.norm4 = nn.BatchNorm1d(32)
        self.act4 = nn.LeakyReLU()
        self.output = nn.Linear(in_features=32, out_features = 3)

    def forward(self, x):
        '''
        Forward pass
        '''
        x = self.norm0(x)
        x = self.linear1(x)
        x = self.norm1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.norm2(x)
        x = self.act2(x)
        x = self.linear3(x)
        x = self.norm3(x)
        x = self.act3(x)
        x = self.linear4(x)
        x = self.norm4(x)
        x = self.act4(x)
        x = self.output(x)

        return x

In [3]:
class ThreeOutputRescaleLog(MLFun.ProcessFunction):
    def __init__(self, firstScale, secondScale, thirdScale):
        self.firstScale = firstScale
        self.secondScale = secondScale
        self.thirdScale = thirdScale
    
    def process(self, input):
        #Process each column by their respective scales
        
        colOne = input[:, 0]
        colOne = colOne / self.firstScale
        colOne = np.log(colOne)
        
        colTwo = input[:, 1]
        colTwo = colTwo / self.secondScale
        colTwo = np.log(colTwo)
        
        colThree = input[:, 2]
        colThree = colThree / self.thirdScale
        colThree = np.log(colThree)
        
        output = torch.stack((colOne, colTwo, colThree), axis = -1)
        
        return output
    
    def invertProcess(self, input):
        
        colOne = input[:, 0]
        colOne = np.exp(colOne)
        colOne = colOne * self.firstScale
        colOne = torch.Tensor(colOne)
        
        colTwo = input[:, 1]
        colTwo = np.exp(colTwo)
        colTwo = colTwo * self.secondScale
        colTwo = torch.Tensor(colTwo)
        
        colThree = input[:, 2]
        colThree = np.exp(colThree)
        colThree = colThree * self.thirdScale
        colThree = torch.Tensor(colThree)
        
        output = torch.stack((colOne, colTwo, colThree), axis = -1)
        output = output.cpu().detach().numpy()
        
        return output
    

# Read in the data

In [4]:
numPoints = 20000
#numPoints = 500
#numPoints = 20000

#filename = 'Data_Fuchs_v_2.2_lambda_um_0.8_points_' + str(numPoints) + '_seed_0.h5'
#filename = 'Dataset/Data_Fuchs_v_2.2_Wright_Pat_Narrow_Range_lambda_um_0.8_points_' + str(numPoints) + '_seed_0.h5'
#filename = 'Dataset/Data_Fuchs_v_2.3_energy_limit_0.01_lambda_um_0.8_points_' + str(numPoints) + '_seed_0.h5'

# filename = 'Dataset/Data_Fuchs_v_2.7_Wright_Pat_Narrow_Range_with_Focal_Dist_energy_limit_0.01_deviation_0.0_lambda_um_0.8_points_' \
#                  + str(numPoints) + '_seed_0.h5'
# filename = 'Dataset/Data_Fuchs_v_2.8_Wright_Pat_Narrow_Range_with_Focal_Dist_energy_limit_0.01_deviation_0.0_' \
#             + 'lambda_um_0.8_points_' + str(numPoints) + '_seed_0.h5'
#filename = 'Dataset/Data_Fuchs_v_2.91_Wright_Pat_intens_focalD_thick_scan_lambda_um_0.8_points_' + str(numPoints) + '_seed_3.h5'
#filename = 'Dataset/Data_Fuchs_v_2.9_Wright_Pat_Narrow_Range_with_Focal_Dist_energy_limit_0.01_deviation_0.0_lambda_um_0.8_points_20000_seed_0.h5'
filename = 'datasets/Energy/fuchs_v3_points_20000_noise_10.h5'


h5File = h5py.File(filename, 'r+')

In [5]:
#Read columns

intens = h5File['Intensity_(W_cm2)']
duration = h5File['Pulse_Duration_(fs)']
thickness = h5File['Target_Thickness (um)']
spotSize = h5File['Spot_Size_(FWHM um)']
focalDist = h5File['Focal_Distance_(um)']
maxEnergy = h5File['Max_Proton_Energy_(MeV)']
totalEnergy = h5File['Total_Proton_Energy_(MeV)']
avgEnergy = h5File['Avg_Proton_Energy_(MeV)']


#Convert columns into numpy arrays
npIntens = np.fromiter(intens, float)
npDuration = np.fromiter(duration, float)
npThickness = np.fromiter(thickness, float)
npSpot = np.fromiter(spotSize, float)
npDist = np.fromiter(focalDist, float)
npMaxEnergy = np.fromiter(maxEnergy, float)
npTotalEnergy = np.fromiter(totalEnergy, float)
npAvgEnergy = np.fromiter(avgEnergy, float)


# #Join all of those arrays into one big numpy array
# npFile = np.dstack((npIntens, npDuration, npThickness, npSpot, npMaxEnergy, npTotalEnergy, npAvgEnergy))
# npFile = npFile.reshape(numPoints, 7)

# npTrain = npFile[:math.floor(.9*numPoints), 0:7]
# #npTest = npFile[math.floor(.9*numPoints):, 0:7]

# npTrain = npFile[:, 0:7]

# #Two input version
# npFile = np.dstack((npIntens, npThickness, npMaxEnergy, npTotalEnergy, npAvgEnergy))
# npFile = npFile.reshape(numPoints, 5)
# npTrain = npFile[:math.floor(.9*numPoints), 0:5]
# npTest = npFile[math.floor(.9*numPoints):, 0:5]

#Three input version
npFile = np.dstack((npIntens, npThickness, npDist, npMaxEnergy, npTotalEnergy, npAvgEnergy))
npFile = npFile.reshape(numPoints, 6)

#If I want to take a subset of the dataset, do it in the line below
numPoints = 20000
np.random.shuffle(npFile)
npFile = npFile[:numPoints]
print(npFile)

npTrain = npFile[:math.floor(.9*numPoints), 0:6]
npTest = npFile[math.floor(.9*numPoints):, 0:6]

# print(npTrain)

#npTrain = npFile[:, 0:5]
#npTrain = npFile[:, 0:6]

#print(npFile.shape)

[[ 2.00246809e+18  9.65419743e+00  8.91527001e+00  2.06861198e-02
   1.49958621e+06  6.98367217e-03]
 [ 2.66931281e+18  1.62030588e+00 -3.32389871e+00  2.76562530e-01
   3.45491626e+07  5.30666304e-02]
 [ 5.23459779e+18  8.88063276e-01  9.85435536e+00  7.25453301e-01
   1.91786607e+08  1.31634819e-01]
 ...
 [ 1.67512102e+18  2.77970861e+00  8.53217179e+00  7.25321819e-02
   4.34108480e+06  1.53091706e-02]
 [ 4.22902816e+18  5.47370265e-01 -5.26398692e+00  7.39598795e-01
   1.45924577e+08  1.41954341e-01]
 [ 2.98502865e+18  9.71667769e+00  5.01767066e+00  5.96168289e-02
   6.25591344e+06  1.52332788e-02]]


In [6]:
#Check to see if there's any negative values that are not focal distance
for index in range(numPoints):
    if(npFile[index, 0] < 0):
        print(index)
        print(npFile[index, 0])
        
    elif(npFile[index, 3] < 0):
        print(index)
        print(npFile[index, 3])
        
    elif(npFile[index, 4] < 0):
        print(index)
        print(npFile[index, 4])
        
    elif(npFile[index, 5] < 0):
        print(index)
        print(npFile[index, 5])

In [7]:
# #filename_test = 'Data_Fuchs_v_2.2_Wright_Pat_Narrow_Range_lambda_um_0.8_points_' + str(numPoints) + '_seed_0.h5'
# #filename_test = 'Dataset/Data_Fuchs_v_2.2_Wright_Pat_Narrow_Range_lambda_um_0.8_points_' + str(100000) + '_seed_1.h5'
# #filename_test = 'Data_Fuchs_v_2.3_energy_limit_0.01_lambda_um_0.8_points_' + str(numPoints) + '_seed_0.h5'
# filename_test = 'Dataset/Data_Fuchs_v_2.7_Wright_Pat_Narrow_Range_with_Focal_Dist_energy_limit_0.01_deviation_0.0_lambda_um_0.8_points_100000_seed_3.h5'

# h5FileTest = h5py.File(filename_test, 'r+')

# #Read columns

# intens = h5FileTest['Intensity_(W_cm2)']
# duration = h5FileTest['Pulse_Duration_(fs)']
# thickness = h5FileTest['Target_Thickness (um)']
# spotSize = h5FileTest['Spot_Size_(FWHM um)']
# focalDist = h5FileTest['Focal_Distance_(um)']
# maxEnergy = h5FileTest['Max_Proton_Energy_(MeV)']
# totalEnergy = h5FileTest['Total_Proton_Energy_(MeV)']
# avgEnergy = h5FileTest['Avg_Proton_Energy_(MeV)']


# #Convert columns into numpy arrays
# npIntens = np.fromiter(intens, float)
# npDuration = np.fromiter(duration, float)
# npThickness = np.fromiter(thickness, float)
# npSpot = np.fromiter(spotSize, float)
# npDist = np.fromiter(focalDist, float)
# npMaxEnergy = np.fromiter(maxEnergy, float)
# npTotalEnergy = np.fromiter(totalEnergy, float)
# npAvgEnergy = np.fromiter(avgEnergy, float)


# # #Join all of those arrays into one big numpy array
# # npFile = np.dstack((npIntens, npDuration, npThickness, npSpot, npMaxEnergy, npTotalEnergy, npAvgEnergy))

# # npFile = npFile.reshape(100000, 7)

# # #npTrain = npFile[:math.floor(.9*numPoints), 0:7]
# # #npTest = npFile[math.floor(.9*numPoints):, 0:7]

# # npTest = npFile[:, 0:7]

# # #Two input version
# # npFile = np.dstack((npIntens, npThickness, npMaxEnergy, npTotalEnergy, npAvgEnergy))
# # npFile = npFile.reshape(100000, 5)

# # npTest = npFile[:, 0:5]

# #Three input version
# npFile = np.dstack((npIntens, npThickness, npDist, npMaxEnergy, npTotalEnergy, npAvgEnergy))
# npFile = npFile.reshape(100000, 6)

# npTest = npFile[:, 0:6]

# print(npFile.shape)

# Prepare our dataset

In [8]:
training_dataset = h5File.create_dataset(name=None, data=npTrain)
test_dataset = h5File.create_dataset(name=None, data=npTest)

In [9]:
#Choose our loss function
loss_function = nn.MSELoss()

In [10]:
#List which epochs we should test

#epochList = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 100, 150, 200, 250]
#epochList = [1, 5, 10, 15, 20, 25, 50, 75, 100, 150, 200]
#epochList = [1, 5, 10, 15, 20, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
#epochList = [1, 5, 10, 15, 20, 25]
#epochList = [1, 5, 10, 15, 20, 30, 35, 40, 45]
#epochList = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
#epochList = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 100]
#epochList = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
#epochList = [1,5, 10, 15, 20, 25, 30, 35]
#epochList = [35]
#epochList = [1, 5, 35]
epochList = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
#epochList = [1, 5, 20]
#epochList = [5, 10, 15]

In [11]:
#Initialize neural network and dataloader
#model1Layer = MultiRegressor1Layer().to('cuda')
#model = MLP().to('cuda')

# modelMSE, modelPercentError, trainMSE, trainPercent, timeList = MLFun.getModelError(model, epochList, loss_function, 
#                                                                                     training_dataset, test_dataset, 
#                                                                                     numInputs = 3, processInputs = MLFun.logFirstCol,
#                                                                                     processTargets = MLFun.logAll)

#Fuchs 2.91 scaling parameters: max = 1e-2, total = 1e5, avg = 1e-3, intens = 1e18
#Fuchs 2.92 scaling parameters: max = 1e0, total = 1e8, avg = 1e-1

maxScale = 1e0
totalScale = 1e8
avgScale = 1e-1
intensScale = 1e19

targetProcess = ThreeOutputRescaleLog(maxScale, totalScale, avgScale)
inputProcess = MLFun.LogFirstColRescaleFun(intensScale)

modelMSE, modelPercentError, trainMSE, trainPercent, timeList, cpuTime = MLFun.getModelError(DJINN, epochList, loss_function, 
                                                                                    training_dataset, test_dataset, 
                                                                                    numInputs = 3, processInputs = inputProcess,
                                                                                    processTargets = targetProcess)

Training with 1 epochs.
Starting epoch 1


AttributeError: 'tuple' object has no attribute 'size'

In [ ]:
def splitErrorList(errorList):
    maxEnergyError = []
    totalEnergyError = []
    avgEnergyError = []

    for element in errorList:
        maxEnergyError.append(element[0])
        totalEnergyError.append(element[1])
        avgEnergyError.append(element[2])
        
    return maxEnergyError, totalEnergyError, avgEnergyError

In [ ]:
# print(model1LayerMSE)
# print(model1LayerPercentError)
# print('\n')
# print(trainMSE)
# print(trainPercent)

maxEnergyMSE, totalEnergyMSE, avgEnergyMSE = splitErrorList(modelMSE)
maxEnergyPercent, totalEnergyPercent, avgEnergyPercent = splitErrorList(modelPercentError)

trainMaxMSE, trainTotalMSE, trainAvgMSE = splitErrorList(trainMSE)
trainMaxPercent, trainTotalPercent, trainAvgPercent = splitErrorList(trainPercent)

#print(trainMaxMSE)
#print(maxEnergyMSE)

# Now plot errors and running time

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Time spent plot
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

plt.plot(epochList, timeList, marker='s')
plt.title("Number of epochs used vs. Time to train neural network")
plt.xlabel("Number of epochs")
plt.ylabel("Time in seconds")


#plt.legend(loc='upper left');
plt.show()

In [ ]:
for epochElement, timeElement in zip(epochList, timeList):
    
    minuteValue = timeElement / 60
    
    print("Number of epochs:", epochElement)
    print("Time spent:", minuteValue, "minutes", '\n')

In [ ]:
#CPU Time spent plot
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

plt.plot(epochList, cpuTime, marker='s')
plt.title("Number of epochs used vs. CPU Time to train neural network")
plt.xlabel("Number of epochs")
plt.ylabel("Time in seconds")


#plt.legend(loc='upper left');
plt.show()

In [ ]:
for epochElement, timeElement in zip(epochList, cpuTime):
    
    minuteValue = timeElement / 60
    
    print("Number of epochs:", epochElement)
    print("CPU Time spent:", minuteValue, "minutes", '\n')

In [ ]:
#Percent Error plot
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

startIndex = 0

plt.plot(epochList[startIndex:], maxEnergyPercent[startIndex:], c='b', marker="s", label='Max Energy')
plt.plot(epochList[startIndex:], totalEnergyPercent[startIndex:], c='r', marker="o", label='Total Energy')
plt.plot(epochList[startIndex:], avgEnergyPercent[startIndex:], c='g', marker='+', label='Average Energy')

data = {'Epochs': epochList[startIndex:], 
        'Max Energy': maxEnergyPercent[startIndex:]}
df = pd.DataFrame(data)

# Write DataFrame to CSV
df.to_csv('data_4hl'+str(batchSize) + str(numPoints)+'.csv', index=False)

plt.title("Testing Data Relative Error")
plt.xlabel("Number of epochs")
plt.ylabel("Percent")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#Percent Error plot for training data
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

plt.plot(epochList[startIndex:], trainMaxPercent[startIndex:], c='b', marker="s", label='Max Energy')
plt.plot(epochList[startIndex:], trainTotalPercent[startIndex:], c='r', marker="o", label='Total Energy')
plt.plot(epochList[startIndex:], trainAvgPercent[startIndex:], c='g', marker='+', label='Average Energy')
plt.title("Training Data Relative Error")
plt.xlabel("Number of epochs")
plt.ylabel("Percent")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#Compare errors of train and test using just the max energy % error

fig = plt.figure()
#ax1 = fig.add_subplot(1,1,1)

plt.plot(epochList[startIndex:], trainMaxPercent[startIndex:], c='b', marker="s", label='Training Data')
plt.plot(epochList[startIndex:], maxEnergyPercent[startIndex:], c='r', marker="s", label='Testing Data')

plt.title("Testing vs. Training Data Error")
plt.xlabel("Number of epochs")
plt.ylabel("Max Energy Percent Error")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#Compare errors of train and test using just the total energy % error

fig = plt.figure()
#ax1 = fig.add_subplot(1,1,1)

plt.plot(epochList[startIndex:], trainTotalPercent[startIndex:], c='b', marker="s", label='Training Data')
plt.plot(epochList[startIndex:], totalEnergyPercent[startIndex:], c='r', marker="s", label='Testing Data')

plt.title("Testing vs. Training Data Error")
plt.xlabel("Number of epochs")
plt.ylabel("Total Energy Percent Error")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#Compare errors of train and test using just the avg energy % error

fig = plt.figure()
#ax1 = fig.add_subplot(1,1,1)

plt.plot(epochList[startIndex:], trainAvgPercent[startIndex:], c='b', marker="s", label='Training Data')
plt.plot(epochList[startIndex:], avgEnergyPercent[startIndex:], c='r', marker="s", label='Testing Data')

plt.title("Testing vs. Training Data Error")
plt.xlabel("Number of epochs")
plt.ylabel("Average Energy Percent Error")
plt.legend(loc='center right')
plt.show()

In [ ]:
for epoch, maxError, totalError, avgError in zip(epochList, maxEnergyPercent, totalEnergyPercent, avgEnergyPercent):
    print("Number of epochs:", epoch)
    print("Max energy percent error:", maxError)
    print("Total energy percent error:", totalError)
    print("Average energy percent error:", avgError, '\n')

In [ ]:
fig = plt.figure(figsize = (12, 4))

plt.subplot(1, 3, 1)
plt.plot(epochList[startIndex:], maxEnergyMSE[startIndex:], marker = 's')
plt.title("Max Energy Mean Squared Error", pad = 20)
plt.xlabel('Number of Epochs')
plt.ylabel('Max Energy Error (MeV)')

plt.subplot(1, 3, 2)
plt.plot(epochList[startIndex:], totalEnergyMSE[startIndex:], marker = 's')
plt.title("Total Energy Mean Squared Error", pad = 20)
plt.xlabel('Number of Epochs')
plt.ylabel('Total Energy Error (MeV)')

plt.subplot(1, 3, 3)
plt.plot(epochList[startIndex:], avgEnergyMSE[startIndex:], marker = 's')
plt.title("Average Energy Absolute Test Error", pad = 20)
plt.xlabel('Number of Epochs')
plt.ylabel('Average Energy Error (MeV)')

In [ ]:
fig = plt.figure(figsize = (12, 4))

plt.subplot(1, 3, 1)
plt.plot(epochList[startIndex:], trainMaxMSE[startIndex:], marker='s')
plt.title("Max Energy Mean Squared Error", pad = 20)
plt.xlabel('Number of Epochs')
plt.ylabel('Max Energy Error (MeV)')

plt.subplot(1, 3, 2)
plt.plot(epochList[startIndex:], trainTotalMSE[startIndex:], marker='s')
plt.title("Total Energy Mean Squared Error", pad = 20)
plt.xlabel('Number of Epochs')
plt.ylabel('Total Energy Error (MeV)')

plt.subplot(1, 3, 3)
plt.plot(epochList[startIndex:], trainAvgMSE[startIndex:], marker='s')
plt.title("Average Energy Absolute Train Error", pad = 20)
plt.xlabel('Number of Epochs')
plt.ylabel('Average Energy Error (MeV)')

In [ ]:
def listSubtract(list1, list2):
    result = []
    
    for x, y in zip(list1, list2):
        difference = x - y
        difference = abs(difference)
        result.append(difference)
        
    
    return result

In [ ]:
#Compare train and test MSE errors on the max energy

fig = plt.figure()
#ax1 = fig.add_subplot(1,1,1)

maxMSEDiff = listSubtract(trainMaxMSE, maxEnergyMSE)

plt.plot(epochList[startIndex:], maxMSEDiff[startIndex:], c='b', marker="s")

plt.title("Max Energy Error MSE Difference Between Training and Testing", pad = 20)
plt.xlabel("Number of epochs")
plt.ylabel("Error Difference")
#plt.legend(loc='upper left')
plt.show()

In [ ]:
#Compare train and test percent errors on the max energy

fig = plt.figure()
#ax1 = fig.add_subplot(1,1,1)

maxPercentDiff = listSubtract(trainMaxPercent, maxEnergyPercent)

plt.plot(epochList[startIndex:], maxPercentDiff[startIndex:], c='b', marker="s")

plt.title("Max Energy Relative Error Difference Between Training and Testing")
plt.xlabel("Number of epochs")
plt.ylabel("Error Difference")
#plt.legend(loc='upper left')
plt.show()

In [ ]:
for epoch, maxError, totalError, avgError in zip(epochList, maxEnergyMSE, totalEnergyMSE, avgEnergyMSE):
    print("Number of epochs:", epoch)
    print("Max energy MSE:", maxError)
    print("Total energy MSE:", totalError)
    print("Average energy MSE:", avgError, '\n')